# Download Dependencies

In [5]:
!pip install -q unsloth==2025.6.8

# Download Datasets and Models

In [6]:
import unsloth

import datasets
import numpy as np

import torch
import trl 
import transformers

print("datasets.__version__", datasets.__version__)
print("numpy.__version__", np.__version__)
print("unsloth.__version__", unsloth.__version__)
print("torch.__version__", torch.__version__)
print("transformers.__version__", transformers.__version__)
print("trl.__version__", trl.__version__)

# datasets.__version__ 3.6.0
# numpy.__version__ 1.26.4
# unsloth.__version__ 2025.6.3
# torch.__version__ 2.7.0+cu126
# transformers.__version__ 4.51.3
# trl.__version__ 0.19.0

datasets.__version__ 3.6.0
numpy.__version__ 1.26.4
unsloth.__version__ 2025.6.8
torch.__version__ 2.7.0+cu126
transformers.__version__ 4.52.4
trl.__version__ 0.19.1


In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import snapshot_download
import os
api = HfApi()

repo_id = "danhtran2mind/Llama-3.2-3B-Reasoning-Vi-Medical-LoRA"
save_path = "./Llama-3.2-3B-Reasoning-Vi-Medical-LoRA"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Download the dataset
snapshot_download(repo_id=repo_id, repo_type="model", local_dir=save_path)

In [ ]:
# To temporary Model hub
from huggingface_hub import HfApi
from huggingface_hub import login
# Initialize API
login("<your_huggingface_token>")  # Replace with your Hugging Face token
api = HfApi()

In [13]:
import numpy as np
from datasets import concatenate_datasets, load_dataset

from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only  
import torch

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

from unsloth import FastModel
from unsloth.chat_templates import get_chat_template

In [ ]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"

lora_rank = 16
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
)

model = FastLanguageModel.get_peft_model(  
    model,  
    r=lora_rank,  
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",  
                    "gate_proj", "up_proj", "down_proj"],  
    lora_alpha=lora_rank,  
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  
    random_state=42,  
    use_rslora=False,  
    loftq_config=None
)

==((====))==  Unsloth 2025.6.8: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.6.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Data Preparetation

In [15]:
train_prompt_style = """Bên dưới là một hướng dẫn mô tả nhiệm vụ, đi kèm với thông tin đầu vào để cung cấp thêm ngữ cảnh.
Hãy viết một phản hồi hoàn thành yêu cầu một cách phù hợp.
Trước khi trả lời, hãy suy nghĩ cẩn thận về câu hỏi và tạo ra một chuỗi suy nghĩ từng bước để đảm bảo một phản hồi logic và chính xác.

### Instruction:
Bạn là một chuyên gia y khoa với kiến thức chuyên sâu về lập luận lâm sàng, chẩn đoán và lập kế hoạch điều trị.
Vui lòng trả lời câu hỏi y khoa sau đây.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [16]:
# EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    inputs = examples["question"]
    complex_cots = examples["context"]
    outputs = examples["answer"]
    texts = []
    for question, cot, response in zip(inputs, complex_cots, outputs):
        # Append the EOS token to the response if it's not already there
        if not response.endswith(tokenizer.eos_token):
            response += tokenizer.eos_token
        text = train_prompt_style.format(question, cot, response)
        texts.append(text)
    return {"text": texts}

In [17]:
from datasets import load_dataset

dataset = load_dataset(
    "tmnam20/ViMedAQA",
    "disease",
    trust_remote_code=True,
)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/501k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/478k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14121 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/784 [00:00<?, ? examples/s]

Map:   0%|          | 0/14121 [00:00<?, ? examples/s]

Map:   0%|          | 0/785 [00:00<?, ? examples/s]

Map:   0%|          | 0/784 [00:00<?, ? examples/s]

In [18]:
# Columns to keep
columns_to_keep = ['question', 'answer', 'context', 'text']

# Columns to remove (all columns except the ones to keep)
columns_to_remove = [col for col in dataset['train'].column_names if col not in columns_to_keep]

# Remove unwanted columns from each split
for split in dataset.keys():
    dataset[split] = dataset[split].remove_columns(columns_to_remove)
# Merge training and validation samples
test_samples = concatenate_datasets([dataset['validation'],
                                         dataset['test']])

# Shuffle the merged dataset with a fixed random seed
training_samples = dataset['train']
test_samples = test_samples.shuffle(seed=42)

print("Number of training samples:", training_samples.num_rows)
print("Number of test samples:", test_samples.num_rows)

Number of training samples: 14121
Number of test samples: 1569


In [19]:
training_samples[0]['text']

'Bên dưới là một hướng dẫn mô tả nhiệm vụ, đi kèm với thông tin đầu vào để cung cấp thêm ngữ cảnh.\nHãy viết một phản hồi hoàn thành yêu cầu một cách phù hợp.\nTrước khi trả lời, hãy suy nghĩ cẩn thận về câu hỏi và tạo ra một chuỗi suy nghĩ từng bước để đảm bảo một phản hồi logic và chính xác.\n\n### Instruction:\nBạn là một chuyên gia y khoa với kiến thức chuyên sâu về lập luận lâm sàng, chẩn đoán và lập kế hoạch điều trị.\nVui lòng trả lời câu hỏi y khoa sau đây.\n\n### Question:\nKhi soi bằng đèn thường, triệu chứng điển hình của viêm mống mắt là gì?\n\n### Response:\n<think>\n5.1 Nhìn về mặt đại thể (khi soi bằng đèn thường) - Đồng tử 2 bên không đều nhau (bên bệnh nhỏ hơn) Có thể phát hiện đồng tử co bên bệnh lý co nhiều hơn bên bình thường, điều này có thể do viêm nhiều dẫn đến dính thành sau mống mắt lại với nhau và với thuỷ tinh thể. Thông thường kích thường đồng tử là 2 – 3 mm, đều 2 bên. - Mủ ở đáy mống mắt (mủ tiền phòng) Đây là hiện tượng các tế bào viêm cùng với chất tiết 

In [20]:
inference_prompt_style = """Bên dưới là một hướng dẫn mô tả một tác vụ, đi kèm với một thông tin đầu vào để cung cấp thêm ngữ cảnh.
Hãy viết một phản hồi để hoàn thành yêu cầu một cách phù hợp.
Trước khi trả lời, hãy suy nghĩ cẩn thận về câu hỏi và tạo một chuỗi suy nghĩ từng bước để đảm bảo phản hồi logic và chính xác.

### Instruction:
Bạn là một chuyên gia y tế có kiến thức chuyên sâu về lập luận lâm sàng, chẩn đoán và lập kế hoạch điều trị.
Vui lòng trả lời câu hỏi y tế sau đây.

### Question:
{}

### Response:
<think>

"""


# Setup Training Arguments

In [ ]:
arguments = TrainingArguments(  
                resume_from_checkpoint="./Llama-3.2-3B-Reasoning-Vi-Medical-LoRA",
                per_device_train_batch_size=8,  
                per_device_eval_batch_size=8,  
                gradient_accumulation_steps=4,  
                eval_strategy="steps",
                eval_steps=100,
                logging_steps=100,
                save_steps=100,
                warmup_steps=30,
                save_total_limit=4,
                num_train_epochs=12, # 5
                # max_steps=50,
                save_strategy="steps",
                metric_for_best_model="eval_loss",
                learning_rate=2e-4,  
                fp16=not is_bfloat16_supported(),  
                bf16=is_bfloat16_supported(),  
                optim="adamw_8bit",  
                weight_decay=0.01,  
                lr_scheduler_type="linear",  
                seed=42,  
                output_dir="./Llama-3.2-3B-Reasoning-Vi-Medical-LoRA",  
                report_to="none",
                load_best_model_at_end=True,  # Load weights with lowest val loss
        		greater_is_better=False,
            )

In [22]:
# Define Trainer
trainer = SFTTrainer(  
    model=model,  
    tokenizer=tokenizer,  
    train_dataset=training_samples,  
    eval_dataset=test_samples,  
    dataset_text_field="text",
    max_seq_length=2048,  
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  
    dataset_num_proc=2,  
    packing=False,  # Can make training 5x faster for short sequences.  
    args=arguments
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/14121 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/1569 [00:00<?, ? examples/s]

# Start train process

In [23]:
trainer.train(resume_from_checkpoint=True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 14,121 | Num Epochs = 12 | Total steps = 5,304
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
5000,0.209300,0.480698
5100,0.187000,0.480475
5200,0.183800,0.477710
5300,0.162600,0.478503


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=5303, training_loss=0.014094636538288132, metrics={'train_runtime': 23329.6269, 'train_samples_per_second': 7.263, 'train_steps_per_second': 0.227, 'total_flos': 1.667517122067204e+18, 'train_loss': 0.014094636538288132})

In [ ]:
model.save_pretrained("Llama-3.2-3B-Reasoning-Vi-Medical-LoRA")  # Local saving
tokenizer.save_pretrained("Llama-3.2-3B-Reasoning-Vi-Medical-LoRA")


('Llama-3.2-3B-Reasoning-Vi-Medical-LoRA/tokenizer_config.json',
 'Llama-3.2-3B-Reasoning-Vi-Medical-LoRA/special_tokens_map.json',
 'Llama-3.2-3B-Reasoning-Vi-Medical-LoRA/chat_template.jinja',
 'Llama-3.2-3B-Reasoning-Vi-Medical-LoRA/tokenizer.json')

# Inference

In [ ]:
# question = training_samples[10]['question']
question = "Tôi bị đau đầu, sốt, đau cứng cổ. Tôi có thể mắc bệnh gì?"
inputs = tokenizer(
    [inference_prompt_style.format(question) + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=2048,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print("Question: ", question)
print(response[0].split("### Response:")[1])